In [1]:
!pip install meteostat

In [2]:
i = 1 # week of tracking data to load

In [3]:
# General Packages
import pandas as pd
import numpy as np
from scipy.interpolate import make_interp_spline
import seaborn as sns
from datetime import date
import datetime
import missingno as msno
import os
import pickle
from tqdm import tqdm
import dataframe_image as dfi
import math
import warnings
from matplotlib import animation
from matplotlib import pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.patches as patches
import ipywidgets as widgets
from datetime import datetime
from meteostat import Point, Daily, Hourly

# PDF Packages
from reportlab.pdfgen.canvas import Canvas
from reportlab.lib import colors
from reportlab.lib.colors import red
from reportlab.lib.colors import black
from reportlab.lib.units import inch
from PyPDF2 import PdfMerger

# Modeling Packages
import xgboost as xg
from sklearn.metrics import mean_squared_error

# My Packages
import cfbd_api_updater

# Package Options
pd.set_option('display.max_columns',200)
pd.set_option('display.max_rows',1000)
warnings.filterwarnings("ignore")
%matplotlib inline

def create_football_field(linenumbers=True,
                          endzones=True,
                          highlight_line=False,
                          highlight_line_number=50,
                          highlighted_name='Line of Scrimmage',
                          fifty_is_los=False,
                          figsize=(12, 6.33)):
    """
    Function that plots the football field for viewing plays.
    Allows for showing or hiding endzones.
    """
    rect = patches.Rectangle((0, 0), 120, 53.3, linewidth=0.1,
                             edgecolor='r', facecolor='darkgreen', zorder=0)

    fig, ax = plt.subplots(1, figsize=figsize)
    ax.add_patch(rect)

    plt.plot([10, 10, 10, 20, 20, 30, 30, 40, 40, 50, 50, 60, 60, 70, 70, 80,
              80, 90, 90, 100, 100, 110, 110, 120, 0, 0, 120, 120],
             [0, 0, 53.3, 53.3, 0, 0, 53.3, 53.3, 0, 0, 53.3, 53.3, 0, 0, 53.3,
              53.3, 0, 0, 53.3, 53.3, 0, 0, 53.3, 53.3, 53.3, 0, 0, 53.3],
             color='white')
    if fifty_is_los:
        plt.plot([60, 60], [0, 53.3], color='gold')
        plt.text(62, 50, '<- Player Yardline at Snap', color='gold')
    # Endzones
    if endzones:
        ez1 = patches.Rectangle((0, 0), 10, 53.3,
                                linewidth=0.1,
                                edgecolor='r',
                                facecolor='blue',
                                alpha=0.2,
                                zorder=0)
        ez2 = patches.Rectangle((110, 0), 120, 53.3,
                                linewidth=0.1,
                                edgecolor='r',
                                facecolor='blue',
                                alpha=0.2,
                                zorder=0)
        ax.add_patch(ez1)
        ax.add_patch(ez2)
    plt.xlim(0, 120)
    plt.ylim(-5, 58.3)
    plt.axis('off')
    if linenumbers:
        for x in range(20, 110, 10):
            numb = x
            if x > 50:
                numb = 120 - x
            plt.text(x, 5, str(numb - 10),
                     horizontalalignment='center',
                     fontsize=20,  # fontname='Arial',
                     color='white')
            plt.text(x - 0.95, 53.3 - 5, str(numb - 10),
                     horizontalalignment='center',
                     fontsize=20,  # fontname='Arial',
                     color='white', rotation=180)
    if endzones:
        hash_range = range(11, 110)
    else:
        hash_range = range(1, 120)

    for x in hash_range:
        ax.plot([x, x], [0.4, 0.7], color='white')
        ax.plot([x, x], [53.0, 52.5], color='white')
        ax.plot([x, x], [22.91, 23.57], color='white')
        ax.plot([x, x], [29.73, 30.39], color='white')

    if highlight_line:
        hl = highlight_line_number + 10
        plt.plot([hl, hl], [0, 53.3], color='yellow')
        plt.text(hl + 2, 50, '<- {}'.format(highlighted_name),
                 color='yellow')
    return fig, ax


def get_venue_data_w_weather(games_df, venue_df):
    
    # Weather Work
    venue_df = games_df[['gameId','gameDate','gameTimeEastern','homeTeamAbbr']].merge(venue_df,
                                                                                 how='left',
                                                                                 left_on=['homeTeamAbbr'],
                                                                                 right_on=['TeamAbbr']).drop('homeTeamAbbr',axis=1)


#     pbar = tqdm(total=len(venue_df))
    for idx,row in venue_df.iterrows():
        _day = int(str(row['gameDate']).split('/')[1])
        _month = int(str(row['gameDate']).split('/')[0])
        _year = int(str(row['gameDate']).split('/')[2])
        _hour = int(str(row['gameTimeEastern']).split(':')[0])
        _min = int(str(row['gameTimeEastern']).split(':')[1])

        _lat = float(row['Stadium Lattitude'])
        _lon = float(row['Stadium Longitude'])

        # Set time period
        start = datetime(_year, _month, _day, _hour, _min)
        end = datetime(_year, _month, _day, _hour+1, _min)

        # Create Point for Vancouver, BC
        location = Point(_lat, _lon)

        # Get hourly data
        data = Hourly(location, start, end)
        data = data.fetch()

        #     temp	The air temperature in °C	Float64
        #     dwpt	The dew point in °C	Float64
        #     rhum	The relative humidity in percent (%)	Float64
        #     prcp	The one hour precipitation total in mm	Float64
        #     snow	The snow depth in mm	Float64
        #     wdir	The average wind direction in degrees (°)	Float64
        #     wspd	The average wind speed in km/h	Float64
        #     wpgt	The peak wind gust in km/h	Float64
        #     pres    The average sea-level air pressure in hPa

        _temp = data['temp'].iat[0]
        _dwpt = data['dwpt'].iat[0]
        _rhum = data['rhum'].iat[0]
        _prcp = data['prcp'].iat[0]
        _snow = data['snow'].iat[0]
        _wdir = data['wdir'].iat[0]
        _wspd = data['wspd'].iat[0]
        _pres = data['pres'].iat[0]

        venue_df.at[idx,'Kickoff_temp'] = _temp
        venue_df.at[idx,'Kickoff_dwpt'] = _dwpt
        venue_df.at[idx,'Kickoff_rhum'] = _rhum
        venue_df.at[idx,'Kickoff_prcp'] = _prcp
        venue_df.at[idx,'Kickoff_snow'] = _snow
        venue_df.at[idx,'Kickoff_wdir'] = _wdir
        venue_df.at[idx,'Kickoff_wspd'] = _wspd
        venue_df.at[idx,'Kickoff_pres'] = _pres

#         pbar.update(1)
#     pbar.close()
    
    venue_df = venue_df.drop(['gameDate','gameTimeEastern','TeamCity','TeamState','Stadium Lattitude',
                              'Stadium Longitude'],axis=1)
    
    return venue_df


def add_off_def_flag(df, plays_df):
    df = df.merge(plays_df[['gameId','playId','possessionTeam']],
                  how='left',
                  on=['gameId','playId'])

    
    df['OFF_DEF_FLAG'] = np.where(df['club']==df['possessionTeam'],'OFF',
                                  np.where(df['club']!='football','DEF','FOOTBALL'))

    return df


def add_closest_opponent_data(df):
    
#     pbar = tqdm(total=len(df))
    for idx,row in df.iterrows():
        if row['OFF_DEF_FLAG']=='OFF':
            cut_df = df[df['frameId']==row['frameId']].reset_index().drop('index',axis=1)
            def_df = cut_df[cut_df['OFF_DEF_FLAG']=='DEF']

            _x = row['x']
            _y = row['y']

            for pidx,prow in def_df.iterrows():
                _raw_dist = (prow['x'] - _x)**2 + (prow['y'] - _y)**2
                def_df.at[pidx,'DISTANCE'] = math.sqrt(_raw_dist)

            def_df = def_df.sort_values(by='DISTANCE',ascending=True).reset_index().drop('index',axis=1)

            _dist = def_df['DISTANCE'].iat[0]
            _def_id = def_df['nflId'].iat[0]
            _def_ori = def_df['o'].iat[0]

            df.at[idx,'DISTANCE_TO_CLOSEST_OPP'] = _dist
            df.at[idx,'CLOSEST_OPP_ID'] = _def_id
            df.at[idx,'DEF_ORIENTATION'] = _def_ori

        elif row['OFF_DEF_FLAG']=='DEF':
            cut_df = df[df['frameId']==row['frameId']].reset_index().drop('index',axis=1)
            off_df = cut_df[cut_df['OFF_DEF_FLAG']=='OFF']

            _x = row['x']
            _y = row['y']

            for pidx,prow in off_df.iterrows():
                _raw_dist = (prow['x'] - _x)**2 + (prow['y'] - _y)**2
                off_df.at[pidx,'DISTANCE'] = math.sqrt(_raw_dist)

            off_df = off_df.sort_values(by='DISTANCE',ascending=True).reset_index().drop('index',axis=1)


            _dist = off_df['DISTANCE'].iat[0]
            _def_id = off_df['nflId'].iat[0]
            _def_ori = off_df['o'].iat[0]

            df.at[idx,'DISTANCE_TO_CLOSEST_OPP'] = _dist
            df.at[idx,'CLOSEST_OPP_ID'] = _def_id
            df.at[idx,'DEF_ORIENTATION'] = _def_ori
        else:
            pass

#         pbar.update(1)
#     pbar.close()
    
    return df


def add_block_engagement_field(df):
    # Distance < 1.25
    # def o must be within 35 degrees +/- of player o + 180
    
#     pbar = tqdm(total=len(df))
    for idx,row in df.iterrows():
        try:
            if row['DISTANCE_TO_CLOSEST_OPP'] <= 1.25:
                _ori = row['o']
                _def_ori = row['DEF_ORIENTATION']

                _ori_diff = abs(_ori - _def_ori)

                if _ori_diff >= 145 and _ori_diff <= 215:
                    df.at[idx,'ENGAGED_IN_BLOCK'] = 1
                else:
                    df.at[idx,'ENGAGED_IN_BLOCK'] = 0 

            else:
                df.at[idx,'ENGAGED_IN_BLOCK'] = 0
        except:
            pass
#         pbar.update(1)
#     pbar.close()

    return df


def add_ball_distance(df):
#     pbar = tqdm(total=len(df))
    for idx,row in df.iterrows():
        cut_df = df[df['frameId']==row['frameId']].reset_index().drop('index',axis=1)

        _ball_df = cut_df[cut_df['club']=='football'].reset_index().drop('index',axis=1)
        _ball_x = _ball_df['x'].iat[0]
        _ball_y = _ball_df['y'].iat[0]

        cut_df = cut_df[cut_df['club']!='football'].reset_index().drop('index',axis=1)

        _x = row['x']
        _y = row['y']

        _dist_to_ball = math.sqrt((_x - _ball_x)**2 + (_y - _ball_y)**2)

        df.at[idx,'DISTANCE_TO_BALL'] = _dist_to_ball

#         pbar.update(1)
#     pbar.close()

    return df  


def add_open_path_to_ball(df):
#     pbar = tqdm(total=len(df))
    for idx,row in df.iterrows():
        if row['OFF_DEF_FLAG'] != 'DEF':
            pass
        else:
            OPEN_PATH_TO_BALL_FLAG = 1

            cut_df = df[df['frameId']==row['frameId']].reset_index().drop('index',axis=1)

            _ball_df = cut_df[cut_df['club']=='football'].reset_index().drop('index',axis=1)
            _ball_x = _ball_df['x'].iat[0]
            _ball_y = _ball_df['y'].iat[0]

            _ply_x = row['x']
            _ply_y = row['y']
            _ply_dist = row['DISTANCE_TO_BALL']

            opp_df = cut_df[cut_df['OFF_DEF_FLAG']=='OFF'].reset_index().drop('index',axis=1)

            # REMOVE BALL CARRIER FROM OPPONENT DATASET
            opp_df = opp_df[opp_df['DISTANCE_TO_BALL']>0.5].reset_index().drop('index',axis=1)

            # LOOP THROUGH OPP DF AND DETERMINE IF ANY OPP IS BETWEEN <_ply_x,_ply_y> and <_ball_x,_ball_y>
            # Angle offset of vectors can be at max 10 degrees for them to be "in the way"
            # Also, distance from opp to ball must be shorter than distance from me to the ball

            # vector from me to ball
            _v1 = (_ply_x - _ball_x, _ply_y - _ball_y)

            for pidx,prow in opp_df.iterrows():
                # opp_loc
                _opp_x = prow['x']
                _opp_y = prow['y']

                _opp_dist = prow['DISTANCE_TO_BALL']

                # vector from me to player
                _v2 = (_ply_x - _opp_x, _ply_y - _opp_y)

                # Unit Vectors
                v1_u = _v1/np.linalg.norm(_v1) # unit vector #3
                v2_u = _v2/np.linalg.norm(_v2) # unit vector #2

                # Angle Between The Vector
                _angle_radians = np.arccos(np.clip(np.dot(v1_u, v2_u), -1.0, 1.0))
                _angle_degrees = _angle_radians * 57.2958

                if _angle_degrees < 10:
                    if _opp_dist < _ply_dist:
                        OPEN_PATH_TO_BALL_FLAG = 0
                else:
                    pass


            df.at[idx,'OPEN_PATH_TO_BALL'] = OPEN_PATH_TO_BALL_FLAG

#         pbar.update(1)
#     pbar.close()

    return df


def add_free_blocker_flag(df):
#     pbar = tqdm(total=len(df))
    for idx,row in df.iterrows():
        cut_df = df[df['frameId']==row['frameId']].reset_index().drop('index',axis=1)

        _ball_df = cut_df[cut_df['club']=='football'].reset_index().drop('index',axis=1)
        _ball_x = _ball_df['x'].iat[0]
        _ball_y = _ball_df['y'].iat[0]

        _ply_x = row['x']
        _ply_y = row['y']
        _ply_dist = row['DISTANCE_TO_BALL']

        opp_df = cut_df[cut_df['OFF_DEF_FLAG']=='OFF'].reset_index().drop('index',axis=1)

        # IDENTIFY BALL CARRIER FROM OPPONENT DATASET
        opp_df = opp_df.sort_values(by='DISTANCE_TO_BALL',ascending=True).reset_index().drop('index',axis=1)
        ball_carrier_df = opp_df.head(1).reset_index().drop('index',axis=1)

        if len(ball_carrier_df)!=0:
            _ball_carrier_id = ball_carrier_df['nflId'].iat[0]

            # Get rid of Ball Carrier from Candidates Of Free Blockers
            opp_df = opp_df[opp_df['nflId']!=_ball_carrier_id].reset_index().drop('index',axis=1)

            # A Free Blocker Must Be Within 4.5 Yards of the Ball Carrier, NOT ENGAGED IN A BLOCK
            opp_df = opp_df[opp_df['ENGAGED_IN_BLOCK']==0]
            opp_df = opp_df[opp_df['DISTANCE_TO_BALL']<=4.5].reset_index().drop('index',axis=1)

            if len(opp_df)!=0:
                FREE_BLOCKER_FLAG = 1
            else:
                FREE_BLOCKER_FLAG = 0
        else:
            FREE_BLOCKER_FLAG = 0

        df.at[idx,'FREE_BLOCKER_W_BALL_CARRIER_FLAG'] = FREE_BLOCKER_FLAG

#         pbar.update(1)
#     pbar.close()

    return df


def engineer_tracking_features(df):
    # Adds All Engineered Tracking Features To The Dataset
    
    # CLOSEST DEF DISTANCE
    df = add_closest_opponent_data(df)

    # BLOCK ENGAGED 
    df = add_block_engagement_field(df)

    # DISTANCE TO BALL
    df = add_ball_distance(df)

    # OPEN PATH TO BALL
    df = add_open_path_to_ball(df)

    # FREE BLOCKER W BALL CARRIER
    df = add_free_blocker_flag(df)
    
    return df


def generate_model_dataset(i):

    # GAME-LEVEL DATA HERE
    games_df = pd.read_csv('../data/raw/kaggle/games.csv')

    # PLAYER LEVEL DATA HERE
    players_df = pd.read_csv('../data/raw/kaggle/players.csv')
    for idx,row in players_df.iterrows():
        players_df.at[idx,'Height_IN'] = int(str(row['height']).split('-')[0])*12 + int(str(row['height']).split('-')[1])
    players_df = players_df.drop('height',axis=1).reset_index().drop('index',axis=1).rename(columns={'Height_IN':'height'})

    # PLAY DATA HERE
    plays_df = pd.read_csv('../data/raw/kaggle/plays.csv')

    for idx,row in plays_df.iterrows():
        if row['possessionTeam'] == row['yardlineSide']:
            # Ball in own half
            plays_df.at[idx,'YardsFromScoring'] = 100 - row['yardlineNumber']
        else:
            plays_df.at[idx,'YardsFromScoring'] = row['yardlineNumber']

    # TARGET DATA HERE
    tackles_df = pd.read_csv('../data/raw/kaggle/tackles.csv')

    # TRACKING DATA
    tracking_df = pd.read_csv(f'../data/raw/kaggle/tracking_week_{i}.csv')

    # Venue Data
    venue_df = pd.read_csv('../data/interim/team_id_listing.csv')
    venue_df = venue_df[['TeamAbbr','TeamCity','TeamState','Stadium Lattitude','Stadium Longitude','Stadium_Domed','Stadium_Turfed']]
    venue_df = get_venue_data_w_weather(games_df, venue_df)


    # Combine Tracking Data Frame-By-Frame w/ Player Details
    df = tracking_df.merge(players_df[['nflId','weight','position','height']],
                           how='left',
                           on=['nflId'])

    # Add in Play-Specific Features
    df = df.merge(plays_df[['gameId','playId','offenseFormation','defendersInTheBox','YardsFromScoring','quarter','down','yardsToGo']],
                  how='left',
                  on=['gameId','playId'])

    # Add in Game-Specific Features
    df = df.merge(games_df[['gameId','homeTeamAbbr']],
                  how='left',
                  on='gameId')

    # Add in Venue Specific Features (Includes Weather)
    df = df.merge(venue_df,
                  how='left',
                  left_on=['gameId','homeTeamAbbr'],
                  right_on=['gameId','TeamAbbr'])

    df = df.drop(['homeTeamAbbr','TeamAbbr'],axis=1)

    # DEFINE WHEN A PLAYER IS ON OFF/DEF
    df = add_off_def_flag(df, plays_df)


    # Add The Engineered Tracking Features
    completed_df = pd.DataFrame()

    pbar = tqdm(total=len(df.drop_duplicates(subset=['gameId','playId'],keep='first')))
    for game_id in df['gameId'].unique().tolist():
        single_game_df = df[df['gameId']==game_id].reset_index().drop('index',axis=1)


        for play_id in single_game_df['playId'].unique().tolist():
            single_play_id = single_game_df[single_game_df['playId']==play_id].reset_index().drop('index',axis=1)

            engineered_df = engineer_tracking_features(single_play_id)
            completed_df = pd.concat([completed_df, engineered_df]).reset_index().drop('index',axis=1)

            pbar.update(1)
    pbar.close()


    # Adding Target Features
    completed_df = completed_df.merge(tackles_df,how='left',on=['gameId','playId','nflId']).reset_index().drop('index',axis=1)

    for col in ['tackle','assist','forcedFumble','pff_missedTackle']:
        completed_df[col] = completed_df[col].fillna(0)

    # Getting Rid of Football as an Object in the Tracking Dataset
    completed_df = completed_df[completed_df['displayName']!='football'].reset_index().drop('index',axis=1)

    # Write to Memory
    completed_df.to_csv(f'../data/processed/ML_Dataset_Week_{i}.csv')
    
    return

In [4]:
for i in range(2,10):
    generate_model_dataset(i)

100%|████████████████████████████████████████████████████████████████████████████| 1179/1179 [3:10:37<00:00,  9.70s/it]


In [9]:
def basic_play_plot(tracking_df, plays_df, games_df, game_id, play_id):

    df = tracking_df[tracking_df['gameId']==game_id]
    df = df[df['playId']==play_id].reset_index().drop('index',axis=1)
    df = df[['frameId','club','displayName','x','y']]

    play_df = plays_df[['gameId','playId','ballCarrierId','quarter','down','yardsToGo','possessionTeam','defensiveTeam','yardlineSide','yardlineNumber','gameClock','preSnapHomeScore','preSnapVisitorScore','offenseFormation','defendersInTheBox','YardsFromScoring','playDescription']]
    play_df = play_df[play_df['gameId']==game_id]
    play_df = play_df[play_df['playId']==play_id].reset_index().drop('index',axis=1)

    game_df = games_df[games_df['gameId']==game_id]
    
    _date = str(game_df['gameDate'].iat[0])
    
    _down = play_df['down'].iat[0]
    _quarter = play_df['quarter'].iat[0]
    _yards_to_go = play_df['yardsToGo'].iat[0]
    _clocktime = play_df['gameClock'].iat[0]

    _off_team = play_df['possessionTeam'].iat[0]
    _def_team = play_df['defensiveTeam'].iat[0]

    _yardline = play_df['yardlineNumber'].iat[0]
    _yardline_side = play_df['yardlineSide'].iat[0]

    _home_score = play_df['preSnapHomeScore'].iat[0]
    _away_score = play_df['preSnapVisitorScore'].iat[0]

    _home_team = game_df['homeTeamAbbr'].iat[0]
    _away_team = game_df['visitorTeamAbbr'].iat[0]

    _play_desc = play_df['playDescription'].iat[0]

    venue_df = pd.read_csv('../data/interim/team_id_listing.csv')
    venue_df = venue_df[['TeamAbbr','TeamCity','TeamState']]
    venue_df = venue_df[venue_df['TeamAbbr']==_home_team].reset_index().drop('index',axis=1)

    _venue_city = venue_df['TeamCity'].iat[0]
    _venue_state = venue_df['TeamState'].iat[0]

    _venue_loc = f"{str(_venue_city)}, {str(_venue_state)}"
    
    if _down == 1: _down = '1st'
    elif _down == 2: down = '2nd'
    elif _down == 3: _down = '3rd'
    elif _down == 4: _down = '4th'

    if _quarter == 1: _quarter = '1st'
    elif _quarter == 2: _quarter = '2nd'
    elif _quarter == 3: _quarter = '3rd'
    elif _quarter == 4: _quarter = '4th'

    plt_title = f"{_home_score}  {_home_team}     {_away_team}  {_away_score}"
    plt_subtitle = f"{_date} | {_venue_loc} | {_quarter}  {_clocktime} | {_off_team} Ball | {_down} & {_yards_to_go} @ {_yardline_side} {_yardline}"

    # Setting Up DFs
    df_ball = df[df['club']=='football'].reset_index().drop('index',axis=1)
    df_no_ball = df[df['club']!='football'].reset_index().drop('index',axis=1)
    df_home = df[df['club']==_home_team].reset_index().drop('index',axis=1)
    df_away = df[df['club']==_away_team].reset_index().drop('index',axis=1)
    
    # Grabbing Colors
    color_df = pd.read_csv('../data/interim/team_id_listing.csv')
    color_df = color_df[['TeamAbbr','Primary_Color_Hex_Code','Secondary_Color_Hex_Code']]
    _home_color = color_df[color_df['TeamAbbr']==_home_team].reset_index()['Primary_Color_Hex_Code'].iat[0]
    _away_color = color_df[color_df['TeamAbbr']==_away_team].reset_index()['Primary_Color_Hex_Code'].iat[0]
    _home_color_2 = color_df[color_df['TeamAbbr']==_home_team].reset_index()['Secondary_Color_Hex_Code'].iat[0]
    _away_color_2 = color_df[color_df['TeamAbbr']==_away_team].reset_index()['Secondary_Color_Hex_Code'].iat[0]
    
    
    def play_plot(i=1):
    
        fig, ax = create_football_field()

        marker_kwargs = {'marker': 'o', 'linestyle': 'None'}

        # Plot The Home Players
        for plyr in df_home['displayName'].unique().tolist():
            df_plyr = df_home[df_home['displayName']==plyr].reset_index().drop('index',axis=1)
            df_plyr = df_plyr[df_plyr['frameId']==i].reset_index().drop('index',axis=1)

            ax.plot([df_plyr['x'].iat[0]], [df_plyr['y'].iat[0]], ms=10, markerfacecolor=_home_color, markeredgecolor=_home_color_2, **marker_kwargs)  # purple

        # Plot The Away Players
        for plyr in df_away['displayName'].unique().tolist():
            df_plyr = df_away[df_away['displayName']==plyr].reset_index().drop('index',axis=1)
            df_plyr = df_plyr[df_plyr['frameId']==i].reset_index().drop('index',axis=1)

            ax.plot([df_plyr['x'].iat[0]], [df_plyr['y'].iat[0]], ms=10, markerfacecolor=_away_color, markeredgecolor=_away_color_2, **marker_kwargs)  # purple

        # PLOT THE BALL
        df_ball_cut = df_ball[df_ball['frameId']==i].reset_index().drop('index',axis=1)
        ax.plot([df_ball_cut['x'].iat[0]], [df_ball_cut['y'].iat[0]], ms=6, markerfacecolor='#594716', markeredgecolor='white',**marker_kwargs)  # purple

        plt.title(f"{plt_subtitle}",fontsize=12)
        fig.suptitle(f"{plt_title}",fontsize=18)


        fig.text(0.5, 0.14, 
                 f"{_play_desc}",
                 horizontalalignment="center")


        home_patch = mpatches.Patch(color=_home_color, label=_home_team)
        away_patch = mpatches.Patch(color=_away_color, label=_away_team)
        fig.legend(handles=[home_patch, away_patch],loc='lower center')
        
        plt.show()
    
        return
    
    widgets.interact(play_plot, 
                     i=widgets.Play(min=1, max=df_ball['frameId'].max()));
    
    return

basic_play_plot(tracking_df = tracking_df, 
                plays_df = plays_df,
                games_df = games_df,
                game_id = 2022091102,
                play_id = 3449)

interactive(children=(Play(value=1, description='i', max=64, min=1), Output()), _dom_classes=('widget-interact…